In [1]:
import json
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import numpy as np
import random
import torch
from torch.optim import Adam
import dgl
from datetime import datetime

from src import *
from pprint import pprint
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

data_dir = './data_passrefinder-p'
output_dir = f"{data_dir}/output/base_passrefinder/tmp/{current_time}"

In [ ]:
from argparse import ArgumentParser
parser = ArgumentParser()

import argparse
from pprint import pprint

parser = argparse.ArgumentParser(description="Parser for model configuration")

parser.add_argument('--feature_file', type=str, default=f"{data_dir}/data/feature_dict.json", help='feature json file')
parser.add_argument('--reuse_rate_file', type=str, default=f"{data_dir}/data/reuse_rate_dict.json", help='reuse rate json file')
parser.add_argument('--sim_file', type=str, default=f"{data_dir}/data/pair_to_sim_induct.json", help='reuse rate json file')
# parser.add_argument('--setting', type=str, required=True, help='graph learning setting: inductive/transductive')
# parser.add_argument('--model_path', type=str, required=True, default="./model/", help='model file path')
parser.add_argument('--output_dir', type=str, default=output_dir, help='output path')
parser.add_argument('--agg_type', type=str, default="attn", help='aggregation type')
# parser.add_argument('--nclient', type=int, default=5, help='number of clients')
# parser.add_argument('--valid', type=float, default=0.2, help='split ratio of validation set')
# parser.add_argument('--test', type=float, default=0.2, help='split ratio of test set')
parser.add_argument('--dropout', type=float, default=0.5, help='dropout ratio')
parser.add_argument('--random_seed', type=int, default=42, help='random seed for initialization')
parser.add_argument('--relu', type=float, default=0.2, help='ReLU threshold')
parser.add_argument('--reuse_th', type=float, default=0.5, help='threshold for reuse')
parser.add_argument('--train_batch_size', type=int, default=2048, help='batch size for training')
parser.add_argument('--eval_batch_size', type=int, default=2048, help='batch size for evaluation')
parser.add_argument('--embed_size', type=int, default=1024, help='size of the embedding layer')
parser.add_argument('--hidden_size', type=int, default=1024, help='size of the hidden layer')
parser.add_argument('--gnn_depth', type=int, default=2, help='depth of the GNN')
parser.add_argument('--max_lr', type=float, default=0.001, help='maximum learning rate')
parser.add_argument('--warmup', type=float, default=0.02, help='warmup ratio for learning rate')
parser.add_argument('--max_epoch', type=int, default=100, help='maximum number of epochs')
# parser.add_argument('--early_stop', type=int, default=40, help='early stopping epochs')
parser.add_argument('--device', type=int, default=0, help='GPU ID')

args = parser.parse_args(args=[])
pprint(args)

torch.manual_seed(args.random_seed)
torch.cuda.manual_seed(args.random_seed)
dgl.seed(args.random_seed)
np.random.seed(args.random_seed)
random.seed(args.random_seed)

In [3]:
def construct_graph_sim(feature_dict, reuse_rate_dict, reuse_th, pair_to_sim):
    print('Constructing graph...')
    
    source_to_id = {s: i for i, s in enumerate(feature_dict.keys())}
    id_to_source = {v: k for k, v in source_to_id.items()}

    id_to_feature = {source_to_id[k]: {'numerical': v['numerical'], 'categorical': v['categorical'], 'url_encoding': v['url_encoding'], 'ip': v['ip'], 'text': v['text']} for k, v in feature_dict.items()}

    score_dict = {}
    label_list = []
    src_list = []
    dst_list = []
    src_list_sim = []
    dst_list_sim = []

    for s1, id1 in source_to_id.items():
        s2_list = reuse_rate_dict[s1]

        for s2, score in s2_list.items():
            id2 = source_to_id.get(s2)
            score = 0 if score < reuse_th else 1

            if id2:
                pair = tuple(sorted((id1, id2)))

                if pair in score_dict:
                    continue

                score_dict[pair] = score

                src_list.append(id1)
                dst_list.append(id2)
                label_list.append(score)

                if pair_to_sim[str(id1)][str(id2)] > 0.6145861872979802:
                    src_list_sim.append(id1)
                    dst_list_sim.append(id2)

    data_dict = {
        ('website', 'user', 'website'): (src_list_sim, dst_list_sim),
        ('website', 'reuse', 'website'): (src_list, dst_list),
    }           

    g = dgl.to_bidirected(dgl.heterograph(data_dict))

    g.ndata['numerical'] = torch.tensor([v['numerical'] for v in id_to_feature.values()])
    g.ndata['categorical'] = torch.tensor([v['categorical'] for v in id_to_feature.values()])
    g.ndata['url_encoding'] = torch.tensor([v['url_encoding'] for v in id_to_feature.values()])
    g.ndata['text'] = torch.tensor([v['text'] for v in id_to_feature.values()])
    g.ndata['ip'] = torch.tensor([v['ip'] for v in id_to_feature.values()], dtype=torch.float)

    g.edata['mask'] = {('website', 'reuse', 'website'): torch.zeros(len(src_list)*2, dtype=torch.float)}
    g.edata['mask'][('website', 'reuse', 'website')][g.edge_ids(src_list, dst_list, etype='reuse')] = torch.tensor(label_list, dtype=torch.float)
    g.edata['mask'][('website', 'reuse', 'website')][g.edge_ids(dst_list, src_list, etype='reuse')] = torch.tensor(label_list, dtype=torch.float)

    g.edata['label'] = {('website', 'reuse', 'website'): torch.zeros(len(src_list)*2, dtype=torch.long)}
    g.edata['label'][('website', 'reuse', 'website')][g.edge_ids(src_list, dst_list, etype='reuse')] = torch.tensor(label_list)
    g.edata['label'][('website', 'reuse', 'website')][g.edge_ids(dst_list, src_list, etype='reuse')] = torch.tensor(label_list)
    
    sys.stdout.write("\033[F")
    print('Constructing graph... Done')

    return g

In [4]:
with open(args.feature_file, 'r') as f:
    feature_dict = json.load(f)

with open(args.reuse_rate_file, 'r') as f:
    reuse_rate_dict = json.load(f)

with open(args.sim_file, 'r') as f:
    pair_to_sim = json.load(f)
    


g = construct_graph_sim(feature_dict, reuse_rate_dict, args.reuse_th, pair_to_sim=pair_to_sim)
g_dict, target_eids_dict = graph_split_base(g, args.random_seed)



# train_loader_list = []
# for g, nodes in zip(g_split['train'], node_split['train']):
#     train_loader_list.append(get_data_loader(g, nodes, args.train_batch_size, args.gnn_depth, suffle=True))
# train_nfeat_list = P.pop_train_node_feature()

# valid_loader = get_data_loader(g_split['valid'], node_split['valid'])
# valid_nfeat = P.pop_node_feature('valid')
# test_loader = get_data_loader(g_split['test'], node_split['test'])
# test_nfeat = P.pop_node_feature('test')


P = PassREfinder_Base(g_dict, target_eids_dict, args)
# P.print_graph()

Constructing graph...
Constructing graph... Done
Splitting graph...
Splitting graph... Done


In [5]:
print(g)

Graph(num_nodes={'website': 9927},
      num_edges={('website', 'reuse', 'website'): 1211942, ('website', 'user', 'website'): 615584},
      metagraph=[('website', 'website', 'reuse'), ('website', 'website', 'user')])


In [7]:
os.makedirs(args.output_dir, exist_ok=True)

with open(os.path.join(args.output_dir, 'arguments.json'), 'w') as f:
    json.dump(vars(args), f)

# args.device = torch.device(f'cuda:{args.device}' if torch.cuda.is_available() else 'cpu')

P.train()

2025-03-27 14:20:37 start


100%|██████████| 24/24 [00:04<00:00,  4.91it/s]


2025-03-27 14:20:47 Epoch [1/100], Train Loss: 0.6976, Valid Loss: 0.6866, Precision: 0.7962, Recall: 0.0217, F1: 0.0423
Save model!


100%|██████████| 24/24 [00:04<00:00,  5.33it/s]


2025-03-27 14:20:56 Epoch [2/100], Train Loss: 0.7069, Valid Loss: 0.6853, Precision: 0.6894, Recall: 0.0036, F1: 0.0072
Save model!


...

2025-03-27 14:35:29 Epoch [98/100], Train Loss: 0.3072, Valid Loss: 0.4964, Precision: 0.8711, Recall: 0.6568, F1: 0.7489


100%|██████████| 24/24 [00:04<00:00,  5.28it/s]


2025-03-27 14:35:38 Epoch [99/100], Train Loss: 0.2969, Valid Loss: 0.4964, Precision: 0.8712, Recall: 0.6568, F1: 0.7489


100%|██████████| 24/24 [00:04<00:00,  4.85it/s]


2025-03-27 14:35:47 Epoch [100/100], Train Loss: 0.3001, Valid Loss: 0.4964, Precision: 0.8711, Recall: 0.6568, F1: 0.7489


In [8]:
%%time
P.model = load_model(os.path.join(P.args.output_dir, 'model.pt'), P.model)
result = P.evaluate()
f1 = result['eval_f1']
precision = result['eval_precision']
recall = result['eval_recall']
print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')

with open(os.path.join(args.output_dir, 'eval_results.json'), 'w') as f:
    json.dump(result, f)

print(f"Saved to {args.output_dir}")

Evaluating...
Precision: 0.8851, Recall: 0.6971, F1: 0.7799
CPU times: user 24min, sys: 6.05 s, total: 24min 6s
Wall time: 6min 57s
